In [116]:
# -*- coding: utf8 -*-
from os.path import expanduser
home = expanduser("~")
repertoire=home+"/Copy/Cours/Bordeaux/L1-UE1/Kalaba-15/05-Classes"
serie=repertoire+"/"
#########################IMPORTS############################################
import codecs, optparse
import re, random
import sys,os,time
import string
import yaml
import ParFuMor as PFM
from ParFuMor import *
import pickle

In [117]:
debug=0
print_glose=True
print_cloze=True
print_radicaux=True
numeros={'1':'Un','2':'Deux','3':'Trois','4':'Quatre','5':'Cinq'}
personnes={'1sg':'UnSg','2sg':'DeuxSg','3sg':'TroisSg','1du':'UnDu','2du':'DeuxDu','3du':'TroisDu','1pl':'UnPl','2pl':'DeuxPl','3pl':'TroisPl'}

commandGrapho="\\newcommand{\\%s}{\\strutgb{0pt}\\grapho{%s}}"
#commandGrapho="\\newcommand{\\%s}{\\strutgb{0pt}{\\dn %s}}"
#commandGrapho=u"\\newcommand{\\%s}{\\strutgb{0pt}{%s}}"
commandPhono=u"\\newcommand{\\%sP}{\\textipa{%s}}"
commandGlose=u"\\newcommand{\\%sG}{\\textGlose{%s}}"

In [118]:
with open(serie+"Phonology.yaml", 'r') as stream:
    phonology=yaml.load(stream)
with open(serie+"Stems.yaml", 'r') as stream:
    stems=yaml.load(stream)
with open(serie+"MorphoSyntax.yaml", 'r') as stream:
    morphosyntax=yaml.load(stream)
with open(serie+"Hierarchie.pkl", 'rb') as input:
   PFM.hierarchieCF = pickle.load(input)
with open(serie+"Lexique.pkl", 'rb') as input:
   PFM.lexique = pickle.load(input)
with open(serie+"Regles.pkl", 'rb') as input:
   PFM.regles = pickle.load(input)

In [119]:
PFM.lexique.lexemes[u'à'].formes

[u'\xe0']

####Définition des segments

In [120]:
consonnes=phonology["consonnes"]
voyelles=phonology["voyelles"]
gabarits=phonology["gabarits"]
derives=phonology["derives"]
nom_classe=phonology["nom_classe"]
nom_apo=phonology["apophonies"]
nom_mut=phonology["mutations"]
syllabes=phonology["syllabes"]

####Attention aux correspondances pour les syllabes
YAML interprète la clé no comme False

In [121]:
def parse_grapho(graphie):
#    chunks=re.findall(r"([ptkbdgmnNfsSvzjrlyv]?[aeiou]?)|[aeiou]|[ptkbdgmnNfsSvzjrlyv]|[.…,;!?:—–()\[\]\/# ""«»<>]", graphie)
#    chunks=re.findall(r"([ptkbdgmnNfsSvzZjwrlyv]?[aeiou]?|[.…,;!?:—–()\[\]\/#""«»<>]+|.*)", graphie,re.U)
    chunks=re.findall(ur"\s+|[ptkbdgmnNfsSvzZjwrlyv]?[aeiou]?", graphie,re.U)
    result=[]
#    print chunks
    for chunk in chunks:
#        print [chunk],syllabes.keys()
        if chunk in syllabes.keys():
            result.append(syllabes[chunk])
        else:
            result.append(chunk)
    return "".join(result)

In [122]:
def parse_cloze(glose):
    if debug: print [glose]
    chunks=re.findall(r"\\cacherGloses{([^}]*)?}|(\w+)", glose,re.UNICODE)
    result=[]
    for chunk in chunks:
        result.extend([x for x in chunk if x!=""])
    return u"%s;"%len(result)+";".join(result)

In [123]:
#grapho=recoder("SviNaNeNNoNN",translit)
#grapho,parse_grapho(grapho)

In [124]:
if "Cas" in morphosyntax:
    casSyntagmes=morphosyntax["Cas"]
else:
    casSyntagmes=""
lexiquePrepositions=[stems["PREP"][x][0] for x in stems["PREP"]]
casPreposition={}
for preposition in lexiquePrepositions:
    prep=preposition.upper()
    if casSyntagmes and prep in casSyntagmes:
        casPreposition[preposition]=casSyntagmes[prep]
    elif casSyntagmes and "PREP" in casSyntagmes:
        casPreposition[preposition]=casSyntagmes["PREP"]
    else:
        casPreposition[preposition]=""
glosePREP={x:u"%s%s"%(x.upper(),cacherGloses("["+casPreposition[x].strip("+").capitalize()+"]")) for x in casPreposition if "+" in casPreposition[x]}

In [125]:
glosePREP

{'avec': u'AVEC\\cacherGloses{[Abs]}',
 'dans': u'DANS\\cacherGloses{[Abs]}',
 'de': u'DE\\cacherGloses{[Gen]}',
 'devant': u'DEVANT\\cacherGloses{[Abs]}',
 'pour': u'POUR\\cacherGloses{[Dat]}',
 'sous': u'SOUS\\cacherGloses{[Abs]}',
 'sur': u'SUR\\cacherGloses{[Abs]}',
 u'\xe0': u'\xc0\\cacherGloses{[Abs]}'}

In [126]:
try:
    __IPYTHON__ 
    ipython=True
except: 
    ipython=False

version=os.path.basename("__file__")
time_stamp='%s' % time.strftime("%y%m%d-%H%M")
print "%% version : "+version
print "%% traitement : "+time_stamp

if ipython or True:
#    lexeme_nom="lexemes.txt"
#    phrase_nom="phrases.txt"
    pass
else:
    parser=optparse.OptionParser()
    parser.add_option("-o", "--out", dest="outfile", action="store_true", help="write to FILE")
    parser.add_option("-c", "--cloze", dest="print_cloze", action="store_true", help="write a CLOZE FILE")
    parser.add_option("-l", "--lexicon", dest="print_lexique", action="store_true", help="append a lexicon")
    parser.add_option("-r", "--roots", dest="print_racines", action="store_true", help="append a root list")

    (options, args) = parser.parse_args()
    lexeme_nom=args[0]
    phrase_nom=args[1]

%% version : __file__
%% traitement : 151018-1610


In [127]:
def recoder(chaine,table,boolUTF8=True):
    if type(chaine)==str:
        temp=unicode(chaine.decode('utf8')).translate(table)
        result=temp
    elif type(chaine)==unicode:
        result=chaine.translate(table)
    if boolUTF8:
        return result
    else:
        return result.encode("utf8")
#translit=string.maketrans(u'iueoaftgzZvjkSpN',u'tgazpHTGZJVkXyxI')

In [128]:
phonoIn =  unicode(phonology["translations"]["grapho"]["in"])
graphoIn = [ord(char) for char in phonoIn]
graphoOut = unicode(phonology["translations"]["grapho"]["out"])
translit = dict(zip(graphoIn, graphoOut))

In [129]:
accentedIn = unicode(phonology["translations"]["deaccent"]["in"])
deaccentIn = [ord(char) for char in accentedIn]
deaccentOut = unicode(phonology["translations"]["deaccent"]["out"])
deaccent = dict(zip(deaccentIn, deaccentOut))

deligatures=phonology["translations"]["deligatures"]

In [130]:
translit

{78: u'N',
 83: u'\xe7',
 90: u'z',
 97: u' ',
 98: u'l',
 100: u'd',
 101: u' ',
 102: u'f',
 103: u'g',
 105: u' ',
 106: u'i',
 107: u'b',
 108: u'c',
 109: u'm',
 110: u'n',
 111: u'u',
 112: u'p',
 114: u'r',
 115: u's',
 116: u't',
 117: u'u',
 118: u'v',
 119: u'u',
 122: u'z'}

In [131]:
tipaIn = unicode(phonology["translations"]["ipa"]["in"])
ipaIn = [ord(char) for char in tipaIn]
ipaOut = unicode(phonology["translations"]["ipa"]["out"])
toipa = dict(zip(ipaIn, ipaOut))

In [132]:
tableaux={}
gloseClozes={}
declarations=[]
for categorie in PFM.lexique.catLexeme:
    if not categorie in tableaux:
        tableaux[categorie]=set()
        gloseClozes[categorie]=[]
    if verbose: print categorie
    for lexeme in PFM.lexique.catLexeme[categorie]:
        if verbose: print PFM.lexique.lexemes[lexeme]
        for case in PFM.lexique.lexemes[lexeme].paradigme.cases:
            caseGlose=case.glose
            if categorie in PFM.categoriesMajeures:
                nom=PFM.lexique.lexemes[lexeme].nom
            else:
                nom=PFM.lexique.lexemes[lexeme].nom.upper()
                gloseCase=case.glose
                if categorie=="PREP" and gloseCase in glosePREP:
                    caseGlose=glosePREP[gloseCase]
                elif categorie=="PREP":
                    caseGlose=caseGlose.upper()
#                print caseGlose
            ref=PFM.modifierGlose(nom,case.sigma,"ref")
            ref=recoder(ref,deaccent)
            for ligature in deligatures:
                ref=ref.replace(ligature,deligatures[ligature])
            for pers in personnes:
                ref=ref.replace(pers,personnes[pers])
            for num in numeros:
                ref=ref.replace(num,numeros[num])
#            phono=case.forme
#            print ("lexeme",lexeme)
            grapho=chaine2utf8(re.sub(ur"\s+",ur"",recoder(parse_grapho(case.forme),translit)))
            if print_radicaux:
                phono=case.detoure.replace(" ","")
            else:
                phono=case.forme.replace(" ","")
#            print [ref,grapho]
            declarations.append(commandGrapho%(ref,grapho))
            declarations.append(commandPhono%(ref,phono))
            declarations.append(commandGlose%(ref,caseGlose))
            if print_glose:
                tableaux[categorie].add("\\"+ref+" & \\"+ref+"P & \\"+ref+"G \\\\")
            else:
                tableaux[categorie].add("\\"+ref+" & \\"+ref+"P & \\blanc{\\"+ref+"G} \\\\")
            if print_cloze:
                vedette=nom.split(".")[0]
                gloses=parse_cloze(case.glose)
                if debug: print [grapho, recoder(phono,toipa)]
                try:
                    cloze=u";".join([ref,vedette,categorie,recoder(phono,toipa),recoder(case.decoupe,toipa),case.sigma,gloses])
                except NameError:
                    cloze=u";".join([ref,vedette,categorie,recoder(phono,toipa),gloses])
                gloseClozes[categorie].append(cloze)
    

In [133]:
if print_radicaux:
    nomDeclaration="Declarations-Radicaux.tex"
    nomTableaux="Tableaux-Radicaux.yaml"
else:
    nomDeclaration="Declarations.tex"
    nomTableaux="Tableaux.yaml"

In [134]:
with codecs.open(serie+nomDeclaration, 'wb', encoding='utf8') as output:
    for declaration in declarations:
#        print type(declaration),declaration
        output.write(declaration+"\n")

In [135]:
if not print_radicaux:
    with codecs.open(serie+"Clozes.txt", 'wb', encoding='utf8') as output:
        for categorie in gloseClozes:
            output.write("#\t"+categorie+"\n#\n#\n")
            for cloze in gloseClozes[categorie]:
                output.write(cloze+"\n")
            output.write("#\n#\n#\n")

In [136]:
yaml.safe_dump(tableaux, file(serie+nomTableaux, 'w'), encoding='utf-8', allow_unicode=True)

In [137]:
#gloseClozes

In [138]:
#declarations